Zapraszam do zabawy <br>
<br>
3 EXP – Wstępna eksploracja <br>
6 EXP – Preprocessing <br>
3 EXP – Przygotowanie podstawowych modeli<br>
6 EXP – Dopracowanie parametrów modeli, ustalenie optymalnych parametrów<br>
2 EXP – Porównanie modeli i wybór najlepszego<br>
6 EXP – Opracowanie warstwy wyjaśniającej (explainability)<br>